# Segmenting and Clustering Toronto Neighbourhoods

## Download and Explore Dataset

In [1]:
!pip install selenium --user
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pandas as pd
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done

# All requested packages already installed.



In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
elems = soup.find_all('table')[0]
table_rows = elems.find_all('tr')

l=[]
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

df = pd.DataFrame(l, columns=['Postcode', 'Borough', 'Neighbourhood'])
df = df.drop(df.index[0])
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n','')
df['Neighbourhood'] = np.where(df.Neighbourhood == 'Not assigned', df.Borough, df.Neighbourhood)
df['Neighbourhood'] = df['Neighbourhood'].replace('Not assigned',np.nan)
df['Borough'] = df['Borough'].replace('Not assigned',np.nan)
df = df.dropna(axis=0, how='any', thresh=None, subset=None)
df = df.reset_index()
del df['index']
df  = df.groupby(['Postcode', 'Borough']).agg({'Neighbourhood': lambda x: ','.join(tuple(x.tolist()))})
df = df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df.shape

(103, 3)

In [5]:
!wget -O load_coordinates.csv http://cocl.us/Geospatial_data

--2020-02-22 20:21:20--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.83, 169.48.113.194, 158.85.108.86
Connecting to cocl.us (cocl.us)|158.85.108.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-02-22 20:21:20--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-22 20:21:21--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-22 20:21:21--  https://ib

In [6]:
df_coord = pd.read_csv('load_coordinates.csv')
df_coord.columns = ['Postcode','Latitude','Longitude']
df_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_final = pd.merge(df, df_coord, on='Postcode')
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
df_set = df_final[df_final['Borough'].str.contains('Toronto')]
df_set = df_set.reset_index()
del df_set['index']
df_set = df_set.iloc[:, [0,1,3,4]]
toronto_data = df_set.copy()
toronto_data.columns = ['Neighborhood', 'Borough', 'Latitude', 'Longitude']
df_set.head()

,Postcode,Borough,Latitude,Longitude
0,M4E,East Toronto,43.676357,-79.293031
1,M4K,East Toronto,43.679557,-79.352188
2,M4L,East Toronto,43.668999,-79.315572
3,M4M,East Toronto,43.659526,-79.340923
4,M4N,Central Toronto,43.728020,-79.388790


## Download Venues

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
# function to get venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=df_set['Postcode'],
                                   latitudes=df_set['Latitude'],
                                   longitudes=df_set['Longitude']
                                  )

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7A
M7Y


In [12]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M4E,5,5,5,5,5,5
M4K,41,41,41,41,41,41
M4L,20,20,20,20,20,20
M4M,42,42,42,42,42,42
M4N,5,5,5,5,5,5
M4P,7,7,7,7,7,7
M4R,23,23,23,23,23,23
M4S,36,36,36,36,36,36
M4T,4,4,4,4,4,4


In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


## Analyze Neighbourhood

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns= cols)

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.20000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,...,0.02439,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.024390
2,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.00,0.023810
4,M4N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.043478
7,M4S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,...,0.00000,0.00,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.00,0.000000


### Neighbourhood with most common venues

In [17]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                  venue  freq
0                 Trail   0.2
1     Health Food Store   0.2
2                   Pub   0.2
3  Other Great Outdoors   0.2
4     Afghan Restaurant   0.0


----M4K----
                    venue  freq
0        Greek Restaurant  0.22
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05


----M4L----
                venue  freq
0         Pizza Place  0.10
1      Sandwich Place  0.10
2  Italian Restaurant  0.05
3             Brewery  0.05
4   Fish & Chips Shop  0.05


----M4M----
                venue  freq
0                Café  0.10
1         Coffee Shop  0.07
2  Italian Restaurant  0.05
3             Brewery  0.05
4           Gastropub  0.05


----M4N----
                        venue  freq
0  Construction & Landscaping   0.2
1          Photography Studio   0.2
2                 Swim School   0.2
3                    Bus Line   0.2
4                        Park   0.2


----M4

In [18]:
# sort function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange( toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues( toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Trail,Health Food Store,Other Great Outdoors,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,M4L,Pizza Place,Sandwich Place,Pub,Burger Joint,Burrito Place,Liquor Store,Park,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,Café,Coffee Shop,Bakery,Brewery,Gastropub,American Restaurant,Italian Restaurant,Yoga Studio,Diner,Bar
4,M4N,Photography Studio,Construction & Landscaping,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


## Cluster Neighbourhood

In [20]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 1, 0, 1, 1, 1, 0, 1], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,43.676357,-79.293031,3,Pub,Trail,Health Food Store,Other Great Outdoors,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M4K,East Toronto,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,M4L,East Toronto,43.668999,-79.315572,1,Pizza Place,Sandwich Place,Pub,Burger Joint,Burrito Place,Liquor Store,Park,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,East Toronto,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Brewery,Gastropub,American Restaurant,Italian Restaurant,Yoga Studio,Diner,Bar
4,M4N,Central Toronto,43.728020,-79.388790,0,Photography Studio,Construction & Landscaping,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [34]:
# create map
latitude = 44
longitude = -79
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,Photography Studio,Construction & Landscaping,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
8,Central Toronto,Playground,Park,Restaurant,Tennis Court,Concert Hall,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,Downtown Toronto,Park,Playground,Trail,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,East Toronto,Pizza Place,Sandwich Place,Pub,Burger Joint,Burrito Place,Liquor Store,Park,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,East Toronto,Café,Coffee Shop,Bakery,Brewery,Gastropub,American Restaurant,Italian Restaurant,Yoga Studio,Diner,Bar
5,Central Toronto,Gym,Park,Department Store,Food & Drink Shop,Sandwich Place,Hotel,Breakfast Spot,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant
6,Central Toronto,Clothing Store,Coffee Shop,Yoga Studio,Gym / Fitness Center,Shoe Store,Salon / Barbershop,Restaurant,Pet Store,Park,Mexican Restaurant
7,Central Toronto,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Gym,Sushi Restaurant,Coffee Shop,Café,Farmers Market,Discount Store
9,Central Toronto,Pub,Coffee Shop,Pizza Place,Light Rail Station,Sports Bar,Supermarket,Sushi Restaurant,Restaurant,Fried Chicken Joint,Liquor Store
11,Downtown Toronto,Park,Café,Restaurant,Coffee Shop,Italian Restaurant,Pub,Bakery,Pizza Place,Pet Store,Sandwich Place
12,Downtown Toronto,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Gym,Hotel,Dance Studio
13,Downtown Toronto,Coffee Shop,Bakery,Pub,Park,Restaurant,Theater,Mexican Restaurant,Breakfast Spot,Café,Spa


### Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,Pool,Garden,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,Pub,Trail,Health Food Store,Other Great Outdoors,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


### Cluster 5

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
